In [1]:
import pandas as pd
from tqdm import tqdm

from uuid import uuid4

from langchain_core.documents import Document
from langchain_qdrant import QdrantVectorStore
from langchain_huggingface import HuggingFaceEmbeddings

from qdrant_client import QdrantClient
from qdrant_client.http.models import Distance, VectorParams

In [4]:
client = QdrantClient(
    path='../db/qdrant_db',
)

collection_name='books-rec-project3'

/tmp/ipykernel_167476/4078310450.py:1: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <books-rec-project> contains 67582 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client = QdrantClient(
/tmp/ipykernel_167476/4078310450.py:1: UserWarning: Local mode is not recommended for collections with more than 20,000 points. Collection <books-rec-project3> contains 67553 points. Consider using Qdrant in Docker or Qdrant Cloud for better performance with large datasets.
  client = QdrantClient(


In [7]:
all_points = []
batch_size = 1000
cursor = None

In [ ]:
print(f'Starting export from local collection: {collection_name}')
while True:
    if cursor is None:
        response = client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            with_payload=True,
            with_vectors=True
        )
    else:
        response = client.scroll(
            collection_name=collection_name,
            limit=batch_size,
            with_payload=True,
            with_vectors=True,
            offset=cursor
        )
        
    points_batch = response[0] if isinstance(response, tuple) else response
    if not points_batch:
        break
    all_points.extend(points_batch)
    
    try:
        _, next_cursor = response
        if next_cursor is None:
            break
        cursor = next_cursor
    except Exception:
        if len(points_batch) < batch_size:
            break
        break
    
print(f"Exported total {len(all_points)} points from local collection.")

In [6]:
client.close()